In [1]:
!pip install corextopic

In [1]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem import WordNetLemmatizer
import pandas as pd
from collections import Counter
import re
import pickle
import string
from sklearn.decomposition import NMF
import spacy
import pandas as pd
import numpy as np
import scipy.sparse as ss
from sklearn.feature_extraction.text import CountVectorizer
from corextopic import corextopic as ct
%matplotlib inline

# Methods to use among files

In [2]:
def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return ' '.join(lemmas)

In [3]:
def only_word(list_):
    """
    This method is to return list of words only in the passed list.
    """
    list_to_return = []
    for ele in list_:
        if len(ele) >= 3:
            list_to_return.append(ele)
    return ' '.join(list_to_return)

In [4]:
def clean_method(text):
    """This method depends on lemmatize_verbs, remove_stopwords and clean_text to apply them on passed parameter"""
    if len(text) <= 100:
        return 'd'
    stopwords = nltk.corpus.stopwords.words('english')
#     text = text.lower() 
    text = text.replace('\\n', ' ').replace('\\r', ' ').replace('\n', ' ').replace('\r', ' ').replace('\xef', ' ').replace('\\xef', ' ')
    text = re.sub('http\S+\s*', ' ', text)
    text = re.sub(r'(.)\1+', r'\1', text)
    text = re.sub('[^a-zA-Z]+', ' ', text)
    text = only_word(text.split())
    text = lemmatize_verbs(text.split())
    text = [word for word in text.split() if word not in stopwords]
    return ' '.join(text)

In [5]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    """...."""
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [6]:
def get_freq_word(TEXT_list):
    """This method get count the frequency of each word in the passed parameter"""
    Vocab_list = ' '.join(TEXT_list)
    temp_dict = Counter(Vocab_list.split()).items()
    freq_df = pd.DataFrame(temp_dict)
    freq_df = freq_df.sort_values(1, ascending = False)
    return freq_df

In [7]:
def write_pickle(file_name, df_to_write):
    """This method is to save the passed dataframe as a pickle file with the passed name"""
    with open(file_name, 'wb') as handle:
        pickle.dump(df_to_write, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [8]:
def read_pickle(file_name):
    """This method is to read the passed file name"""
    with open(file_name , 'rb') as handle:
        return pickle.load(handle)

In [9]:
def write_pickle(file_name, df_to_write):
    """This method is to save the passed dataframe as a pickle file with the passed name"""
    with open(file_name, 'wb') as handle:
        pickle.dump(df_to_write, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [10]:
def remove_domain_stop_words(stop_words, df_edit):
    """Remove domain specific stop words from the passed parameter"""
    df_after = df_edit.copy()
    for i in range(len(df_after)):
        for word in stop_words:
            pattern = r'\b' + word + r'\b'
            if isinstance(df_after.TEXT[i], str): # Check if the value is a string
                new_text = re.sub(pattern,'', df_after.TEXT[i])
                df_after.loc[i, 'TEXT'] = new_text
    return df_after

# Now Read Data from CSV file

In [11]:
data = pd.read_csv('resume_dataset.csv')
data2 = pd.read_csv('resume_dataset2.csv')
data = pd.concat([data, data2])

# Drop unnecessary column

In [12]:
# Drop Category and ID columns since we want to apply unsupervised learning procedures
data = data.drop(['Category', 'ID'],axis=1) 
data.columns = ['TEXT']
data.index = range(0, len(data))

In [13]:
data

,TEXT
0,Skills * Programming Languages: Python (pandas...
1,Education Details \r\nMay 2013 to May 2017 B.E...
2,"Areas of Interest Deep Learning, Control Syste..."
3,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,"Education Details \r\n MCA YMCAUST, Faridab..."
...,...
1383,"b""Free Flight Attendant Resume\nDarlene Flint\..."
1384,b'Corporate Flight Attendant Resume\nCAITLIN F...
1385,"b'MAJOR CONRAD A. PREEDOM\n2354 Fairchild Dr.,..."
1386,b'STACY SAMPLE\n\n702 800-0000 cell\n\n0000@em...


# Explore data

In [14]:
## this step is to display all text in each row
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [15]:
data.sample(2, random_state = 1)

TEXT
255  b'Xochil Herrera Scheer\n\n1307 S. WOLF ROAD\nHILLSIDE, ILLINOIS 60162\nCONTACTME@XOCHIL.COM\n708.790.5190\nlinkedin.com/in/xochil\n\nSkills\n\nFASHION TECHNICAL: Product Development for Apparel (Sportswear, Activewear, Knits) and Accessories, Pattern Making,\nGrading, Sewing, Technical Design, Fabric Sourcing and Development, Textile Knowledge, Leather and Fur Construction,\nAdvanced Tailoring and Garment Redesign\nCOMPUTER: Adobe Photoshop and Illustrator, Optitex Pattern Design, Gerber Accumark Pattern Design, Microsoft Office\nMANAGEMENT/CUSTOMER SERVICE: Managing employees and contractors, project schedules and operations, sales, inventory\nand pricing. Experienced in product development, merchandising, retail buying, and event planning.\n\nFreelance Experience\n\n2004-Present\nXochil Herrera Scheer Fashion Design Services: Pattern making and sample sewing for designers and\nApparel companies. Designed sports wear and evening wear for clients under independent label. Services includes pattern making,\ntechnical design, fabric sourcing, line development, production samples, sewing and tailoring.\nClients/Projects: Diamond MMA, Rayjus Outdoors, Borris Powell, Glass House Shirtmakers, Lauren Lein, Crescendo Apparel,\nBlake Standard, Doc Froc, Cotton Candy Couture, Smashing Golf & Tennis, The Giving Bride, Ellie Day, Skate Armor, Pine & Plastic,\nCasual Friday Apparel\n2009-2011\nThe Image Studios: Tailoring and advanced clothing re-design services for professional client wardrobes.\nWorked directly with stylists and image consultants to achieve strong results for client\'s "personal branding."\n2005\nHouse of Blues - Chicago: Designed and constructed cocktail uniforms (Oct. 2005) and women\xe2\x80\x99s shirt\n(Jan. 2005) for restaurant employees and for sale in the Company Store.\n\nJob Experience\n\nThe Apparel Agency\nChicago, IL\nProduct Development Manager\nWorks together with a team of professionals offering product development services, business and brand strategy, and production\nservices. Works directly with clients, comprehensively throughout each stage of development. Work includes fabric and material\nsourcing and development, technical design, pattern making, and product design. Apparel and accessory sewn products are\nprepared for prototyping and production. Manages support staff, internship program, and works directly with fabric mills, vendors\nand factory sub-contractors. Attends domestic and international trade shows, including TexWorld USA, Premiere Vision, Indigo,\nKingpins, TexWorld Paris, Premiere Vision Plur/El Paris, Modamont, TexProcess Americas.\n2012-2014\n\nClients/Projects: FOMO Activewear, Hank USA, Moxie Cycling, Robyn Glaser Home, 1420 Collection, Stella Blu Clothing,\nAll The Numbers, Molly Ades Maternity, Iguana Med, Michael Alan Stein, Kore Extreme, Hershey\'s Chocolate World\nTurley Road\nChicago, IL\nPattern Maker\nWorked with owner and creative director to design new styles, and revised existing patterns. Managed pattern library. Graded all\npatterns to specified sizes, and cut for small production. Sewed fit samples for new styles and worked with sewing contractor.\n2009-2012\n\nEliana Lily Chicago and Blo Salon and Spa\nChicago, IL\nDesign and Restyling Manager; promoted to General Manager\nManaged Custom Clothing, Restyling and Handbag department, including design, patterns, fabric selection, construction and\nfinishing. Handled vendor relations, inventory, pricing, customer service. Assisted in retail buying. Oversaw up to 15 employees and\ninternship program. Involved in event planning partnering with organizations, charities, and fashion/trunk shows including Lincoln\nPark Chamber of Commerce, Poor Little Rich Girls, Yelp.com, and In Style magazine. Created all promotional materials including\nsignage, posters, flyers, mailings.\n2006-2009\n\nThe Illinois Institute of Art - Chicago\nChicago, IL\nFashion Department Tutor\nHelped students in sewing, pattern making, draping a

In [16]:
data.head(2)

TEXT
0  Skills * Programming Languages: Python (pandas, numpy, scipy, scikit-learn, matplotlib), Sql, Java, JavaScript/JQuery. * Machine learning: Regression, SVM, NaÃ¯ve Bayes, KNN, Random Forest, Decision Trees, Boosting techniques, Cluster Analysis, Word Embedding, Sentiment Analysis, Natural Language processing, Dimensionality reduction, Topic Modelling (LDA, NMF), PCA & Neural Nets. * Database Visualizations: Mysql, SqlServer, Cassandra, Hbase, ElasticSearch D3.js, DC.js, Plotly, kibana, matplotlib, ggplot, Tableau. * Others: Regular Expression, HTML, CSS, Angular 6, Logstash, Kafka, Python Flask, Git, Docker, computer vision - Open CV and understanding of Deep learning.Education Details \r\nData Science Assurance AssociateData Science Assurance Associate - Ernst & Young LLPSkill Details \r\nJAVASCRIPT- Exprience - 24 months\r\njQuery- Exprience - 24 months\r\nPython- Exprience - 24 monthsCompany Details \r\ncompany - Ernst & Young LLP\r\ndescription - Fraud Investigations and Dispute Services   Assurance\r\nTECHNOLOGY ASSISTED REVIEW\r\nTAR (Technology Assisted Review) assists in accelerating the review process and run analytics and generate reports.\r\n* Core member of a team helped in developing automated review platform tool from scratch for assisting E discovery domain, this tool implements predictive coding and topic modelling by automating reviews, resulting in reduced labor costs and time spent during the lawyers review.\r\n* Understand the end to end flow of the solution, doing research and development for classification models, predictive analysis and mining of the information present in text data. Worked on analyzing the outputs and precision monitoring for the entire tool.\r\n* TAR assists in predictive coding, topic modelling from the evidence by following EY standards. Developed the classifier models in order to identify "red flags" and fraud-related issues.\r\n\r\nTools & Technologies: Python, scikit-learn, tfidf, word2vec, doc2vec, cosine similarity, NaÃ¯ve Bayes, LDA, NMF for topic modelling, Vader and text blob for sentiment analysis. Matplot lib, Tableau dashboard for reporting.\r\n\r\nMULTIPLE DATA SCIENCE AND ANALYTIC PROJECTS (USA CLIENTS)\r\nTEXT ANALYTICS - MOTOR VEHICLE CUSTOMER REVIEW DATA * Received customer feedback survey data for past one year. Performed sentiment (Positive, Negative & Neutral) and time series analysis on customer comments across all 4 categories.\r\n* Created heat map of terms by survey category based on frequency of words * Extracted Positive and Negative words across all the Survey categories and plotted Word cloud.\r\n* Created customized tableau dashboards for effective reporting and visualizations.\r\nCHATBOT * Developed a user friendly chatbot for one of our Products which handle simple questions about hours of operation, reservation options and so on.\r\n* This chat bot serves entire product related questions. Giving overview of tool via QA platform and also give recommendation responses so that user question to build chain of relevant answer.\r\n* This too has intelligence to build the pipeline of questions as per user requirement and asks the relevant /recommended questions.\r\n\r\nTools & Technologies: Python, Natural language processing, NLTK, spacy, topic modelling, Sentiment analysis, Word Embedding, scikit-learn, JavaScript/JQuery, SqlServer\r\n\r\nINFORMATION GOVERNANCE\r\nOrganizations to make informed decisions about all of the information they store. The integrated Information Governance portfolio synthesizes intelligence across unstructured data sources and facilitates action to ensure organizations are best positioned to counter information risk.\r\n* Scan data from multiple sources of formats and parse different file formats, extract Meta data information, push results for indexing elastic search and created customized, interactive dashboards using kibana.\r\n* Preforming ROT Analysis on the data which give information of data which helps identify conten

In [17]:
data.tail(2)

TEXT
1386                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1388 entries, 0 to 1387
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TEXT    1388 non-null   object
dtypes: object(1)
memory usage: 11.0+ KB


In [19]:
#Checking for nulls
data.isnull().sum()

TEXT    0
dtype: int64

In [20]:
#Checking for duplicates
print((data.duplicated('TEXT').sum()))

131


In [21]:
#drop duplicates
data.drop_duplicates('TEXT', inplace = True)

# Start Resume Text cleaning 

In [22]:
def clean_method(TEXT):
    """
    Clean text by removing special characters and returning 'd' for empty strings.
    """
    if isinstance(TEXT, str):
        # Remove special characters from the text
        cleaned_text = re.sub('[^a-zA-Z0-9 \n\.]', '', TEXT)
        # Check if the text is empty after cleaning
        if not cleaned_text.strip():
            return 'd'
        else:
            return cleaned_text.lower()
    else:
        return 'd'

In [23]:
# clean text and any empty text will return 'd'
data.index = range(0, len(data))
data['TEXT'] = data['TEXT'].apply(lambda x: clean_method(x))

In [24]:
# Remove empty text
data = data[~(data.TEXT == 'd')]

In [25]:
data.head(2) # data after cleaning

TEXT
0  skills  programming languages python pandas numpy scipy scikitlearn matplotlib sql java javascriptjquery.  machine learning regression svm nave bayes knn random forest decision trees boosting techniques cluster analysis word embedding sentiment analysis natural language processing dimensionality reduction topic modelling lda nmf pca  neural nets.  database visualizations mysql sqlserver cassandra hbase elasticsearch d3.js dc.js plotly kibana matplotlib ggplot tableau.  others regular expression html css angular 6 logstash kafka python flask git docker computer vision  open cv and understanding of deep learning.education details \ndata science assurance associatedata science assurance associate  ernst  young llpskill details \njavascript exprience  24 months\njquery exprience  24 months\npython exprience  24 monthscompany details \ncompany  ernst  young llp\ndescription  fraud investigations and dispute services   assurance\ntechnology assisted review\ntar technology assisted review assists in accelerating the review process and run analytics and generate reports.\n core member of a team helped in developing automated review platform tool from scratch for assisting e discovery domain this tool implements predictive coding and topic modelling by automating reviews resulting in reduced labor costs and time spent during the lawyers review.\n understand the end to end flow of the solution doing research and development for classification models predictive analysis and mining of the information present in text data. worked on analyzing the outputs and precision monitoring for the entire tool.\n tar assists in predictive coding topic modelling from the evidence by following ey standards. developed the classifier models in order to identify red flags and fraudrelated issues.\n\ntools  technologies python scikitlearn tfidf word2vec doc2vec cosine similarity nave bayes lda nmf for topic modelling vader and text blob for sentiment analysis. matplot lib tableau dashboard for reporting.\n\nmultiple data science and analytic projects usa clients\ntext analytics  motor vehicle customer review data  received customer feedback survey data for past one year. performed sentiment positive negative  neutral and time series analysis on customer comments across all 4 categories.\n created heat map of terms by survey category based on frequency of words  extracted positive and negative words across all the survey categories and plotted word cloud.\n created customized tableau dashboards for effective reporting and visualizations.\nchatbot  developed a user friendly chatbot for one of our products which handle simple questions about hours of operation reservation options and so on.\n this chat bot serves entire product related questions. giving overview of tool via qa platform and also give recommendation responses so that user question to build chain of relevant answer.\n this too has intelligence to build the pipeline of questions as per user requirement and asks the relevant recommended questions.\n\ntools  technologies python natural language processing nltk spacy topic modelling sentiment analysis word embedding scikitlearn javascriptjquery sqlserver\n\ninformation governance\norganizations to make informed decisions about all of the information they store. the integrated information governance portfolio synthesizes intelligence across unstructured data sources and facilitates action to ensure organizations are best positioned to counter information risk.\n scan data from multiple sources of formats and parse different file formats extract meta data information push results for indexing elastic search and created customized interactive dashboards using kibana.\n preforming rot analysis on the data which give information of data which helps identify content that is either redundant outdated or trivial.\n preforming fulltext search analysis on elastic search with predefined methods which can tag as pii personally identifiable information so

In [26]:
data.tail(2) # data after cleaning

TEXT
1255  bmajor conrad a. preedomn2354 fairchild dr. suite 6h126nusaf academy co 808405099n719 3338259nconrad.preedomusafa.edunteaching experiencenassistant professor of management director of curriculum us air force academy co 2009presentncoordinated over 200 classes annually for 35 faculty members and over 600 cadets developed curriculum for advancednmanagement courses provided classroom instruction to 12 sections of cadets over 5 semesters in multiple upperlevelncourses provided focused professional mentorship to more 30 cadets.ndiamond star t52a instructor pilot us air force academy co 2008presentnperformed instructor pilot duties in a singleengine diamond star da40 trainer aircraft provided instruction tonseasoned air force pilots as well as us air force academy cadets both in and out of the cockpit.ncessna t37b instructor pilot laughlin air force base tx 20002003nperformed instructor pilot duties at the us air forcexe2x80x99s largest pilot training base in air forcexe2x80x99s primary jet trainernprovided both classroom instruction and flight instruction in all aspects of flying to include traffic patternslandingsnaerobatic flight spin training instrument flying formation flying and both day and night cross country navigation.nschweitzer 233 instructor pilot us air force academy co 19951998nperformed instructor pilot duties in the us air force academyxe2x80x99s soar for all program provided flightline and sailplanencockpit instruction to 5 semesters of cadets with a focus on aerotow primary maneuvering traffic patterns and landing.nflying experiencen2009 xe2x80x93 present 160 flight hours145 instructor pilot hours in diamond star da40 usaf t52n2004 xe2x80x93 2007n442 hours45 flight lead hours in mcdonnell douglas f15e strike eaglen2003 xe2x80x93 2004n19 hours in northrop at38bn2000 xe2x80x93 2003n1311 flight hours1051 instructor pilot hours in cessna t37bn1999 xe2x80x93 2000n26 flight hours in northrop t38an1995 xe2x80x93 1998n250 flight hours175 instructor pilot hours in schweitzer 233 usaf tg4nother aircraft flown include the slingsby t3a firefly cessna t41cd and the cessna 172.nusaf duty historyn2008 xe2x80x93 present assistant professor of management director of curriculum us air force academy con2007 xe2x80x93 2008ndirector of staff seventh air force operationsplans a3  a5 osan air base republic of korean2004 xe2x80x93 2007ncommander of plans flight f15e multiship flight lead seymour johnson air force base ncn2003 xe2x80x93 2004nf15e student pilot seymour johnson air force base ncn2000 xe2x80x93 2003nexecutive officer 47th operations group t37b instructor pilot laughlin air force base txn1999 xe2x80x93 2000nus air force pilot training student laughlin air force base txnformal educationnmba business administration  management university of pittsburgh pa 1999 gpa 3.88nbs operations research us air force academy co 1998 gpa 3.72nprofessional military educationnair command and staff college xe2x80x93 2008nsquadron officer school xe2x80x93 2005nratings and certificationsncertified flight instructor federal aviation administration xe2x80x93 2010nsenior pilot united states air force xe2x80x93 2007ncommercial pilot federal aviation administration xe2x80x93 2000npilot united states air force xe2x80x93 2000nbasic parachutist united states air force xe2x80x93 1995nnx0chonor and awardsnfield grade officer of the quarter division level dept of management us air force academy co xe2x80x93 2010noutstanding academy educator award dept of management us air force academy co xe2x80x93 2010nfield grade officer of the quarter division level dept of management us air force academy co xe2x80x93 2009nf. theodore helmer teaching award department of management us air force academy co xe2x80x93 2008ncompany grade officer of the quarter 4th operations support squadron seymour johnson air force base nc xe2x80x93 2006nf15e leadership award 334th fighter squadron seymour johnson air force base nc xe2x80x93 2004ninstructor pilot of the year 84th 

In [27]:
data.sample(2) 

,TEXT
673,babstractor resume templatenglenda gaken876 jacskonnmississippi 76889nphone 878 908909nggtyycmail.comnobjectivenanimalcaring individual seeking for a position of animal trainer in a standard zoonsummary of qualificationnxefx82xb7nnwith almost five years experience in training horses lions and tigersnnxefx82xb7nnknowledgeable in animal health diet and behaviornnxefx82xb7nnvery responsible and with deep concern of the animals under supervision and carennxefx82xb7nnable in operating equipment for animal health assistance such as tracking systemsnclippers and machines for daily hygienennxefx82xb7nnvery dependable and a great team player.nncareer experiencejob historyn2008 xe2x80x93 2011 animal trainer mississippi zoonxefx82xb7nnassess animals under supervision and kept records of their general wellbeingnnxefx82xb7nnidentified behaviors that need to be tamed and created a training program for each animalnunder care xe2x80x93 horses and lionsnnxefx82xb7nnarranged for mating of stallions and mares and assist in foal deliverynnxefx82xb7nnfed the animals and worked them up for their daily exercisesnnxefx82xb7nntrained horses for synchronized performances and for enduring performances with anhuman entertainernn20042008 zookeeper monroe zoonxefx82xb7nnmaintained zoo cleanliness by implementing programs with supervisionnnx0cxefx82xb7nnobserved animals identified health problems and immediately sought for treatment with thenzoo veterinariannnxefx82xb7nncreated a diet program for the animals and maintained records and files for each zoo areanneducationn19982002 bachelor of science in zoology university of mississippinprofessional reference will be furnished upon request
749,bfree sales resume templatennadam duffyn56th fire streetnfountain lake florida 02976nphone 65467866785nxxxxgmail.comnobjectivena home dxc3xa9cor sales representative that will utilize experience and expertise within thenresources provided. desire a job with growth potential.nsummary of qualificationninnovative sales professional with 9 years of sales experience of selling home productsnxefx82xb7nngood at selling furniture pricing strategies and dealing with customers.nxefx82xb7 adopt a team approach to achieve the objectives.nxefx82xb7 quick learning ability in order to grasp new things and to put ideas in application.nxefx82xb7 excellent communication skills and attitude towards every customerncareer experiencejob historyn2004 till presentnused furniture sales manager at jacksonville furniture in floridanxefx82xb7nnpurchased furniture from auctions and home sales refurnish them and sell at the reasonablenprofit. hired trained and mentored 10 sales people.n20022004nsales manager fresh stock inc floridannxefx82xb7nnsold the furniture by having direct contact witnachievementsnnxefx82xb7nxefx82xb7nnbest sales representative award of the yearnfixed increment added in the fixed salary as a reward for constant two years of sincerenservicennx0cxefx82xb7nnbonus reward for increasing product demand by good promotion schemesneducationn20002004nbachelors of arts university of floridanprofessional reference will be provided upon request


In [28]:
data.shape

(1257, 1)

In [29]:
import re
data['TEXT'] =data['TEXT'].apply(lambda x: re.sub(r'\d+', '', x))

In [30]:
# remove punctuation characters from the TEXT column
data['TEXT'] = data['TEXT'].str.replace('[{}]'.format(string.punctuation), '')

In [31]:
data['TEXT'] = data['TEXT'].str.replace('\n', '', regex=True)

In [32]:
data.head()

TEXT
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  skills  programming languages python pandas numpy scipy scikitlearn matplotlib sql java javascriptjquery.  machine learning regression svm nave bayes knn random forest decision trees boosting techniques cluster analysis word embedding sentiment analysis natural language processing dimensionality reduction topic modelling lda nmf pca  neural nets.  database visualizations mysql sqlserver cassandra hbase elasticsearch d.js dc.js plotly kibana matplotlib ggplot tableau.  others regular expression html css angular  logstash kafka python flask git docker computer vision  open cv and understanding of deep learning.education details data science assurance associatedata science assurance associate  ernst  young llpskill details javascript exprience   monthsjquery exprience   monthspython exprience   monthscompany details company  ernst  young llpdescription  fraud investigations and dispute services   assurancetechnology assisted reviewtar technology assisted review assists in accelerating the review process and run analytics and generate reports. core member of a team helped in developing automated review platform tool from scratch for assisting e discovery domain this tool implements predictive coding and topic modelling by automating reviews resulting in reduced labor costs and time spent during the lawyers review. understand the end to end flow of the solution doing research and development for classification models predictive analysis and mining of the information present in text data. worked on analyzing the outputs and precision monitoring for the entire tool. tar assists in predictive coding topic modelling from the evidence by following ey standards. developed the classifier models in order to identify red flags and fraudrelated issues.tools  technologies python scikitlearn tfidf wordvec docvec cosine similarity nave bayes lda nmf for topic modelling vader and text blob for sentiment analysis. matplot lib tableau dashboard for reporting.multiple data science and analytic projects usa clientstext analytics  motor vehicle customer review data  received 

In [33]:
# define a regular expression pattern to match merged words
pattern = r'([a-z])([A-Z])'

# apply the pattern to the column and add a space between merged words
data['TEXT'] = data['TEXT'].apply(lambda x: re.sub(pattern, r'\1 \2', x))

In [34]:
data.head()

TEXT
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  skills  programming languages python pandas numpy scipy scikitlearn matplotlib sql java javascriptjquery.  machine learning regression svm nave bayes knn random forest decision trees boosting techniques cluster analysis word embedding sentiment analysis natural language processing dimensionality reduction topic modelling lda nmf pca  neural nets.  database visualizations mysql sqlserver cassandra hbase elasticsearch d.js dc.js plotly kibana matplotlib ggplot tableau.  others regular expression html css angular  logstash kafka python flask git docker computer vision  open cv and understanding of deep learning.education details data science assurance associatedata science assurance associate  ernst  young llpskill details javascript exprience   monthsjquery exprience   monthspython exprience   monthscompany details company  ernst  young llpdescription  fraud investigations and dispute services   assurancetechnology assisted reviewtar technology assisted review assists in accelerating the review process and run analytics and generate reports. core member of a team helped in developing automated review platform tool from scratch for assisting e discovery domain this tool implements predictive coding and topic modelling by automating reviews resulting in reduced labor costs and time spent during the lawyers review. understand the end to end flow of the solution doing research and development for classification models predictive analysis and mining of the information present in text data. worked on analyzing the outputs and precision monitoring for the entire tool. tar assists in predictive coding topic modelling from the evidence by following ey standards. developed the classifier models in order to identify red flags and fraudrelated issues.tools  technologies python scikitlearn tfidf wordvec docvec cosine similarity nave bayes lda nmf for topic modelling vader and text blob for sentiment analysis. matplot lib tableau dashboard for reporting.multiple data science and analytic projects usa clientstext analytics  motor vehicle customer review data  received 

# Remove domain specific stopwords

In [35]:
freq_words = get_freq_word(list(data.TEXT))

In [36]:
freq_words.columns = ['WORD', 'COUNT']
freq_words

,WORD,COUNT
70,and,36926
72,of,24168
103,the,17842
143,to,15360
101,in,14512
...,...,...
56951,signs.nthrough,1
56949,sangnsongs,1
56948,marched,1
56947,peacefulndemonstration.,1


In [37]:
freq_words.to_csv('freq_words_3.csv')

In [38]:
words_df = pd.read_csv('freq_words_3_after_delete_col.csv')

In [39]:
# replace 1s with 0s and vice versa in the DELETE column
words_df['DELETE'] = words_df['DELETE'].apply(lambda x: 1 - x)

# write the modified dataframe back to a CSV file
words_df.to_csv('modified_freq_words_3_after_delete_col.csv', index=False)

In [40]:
words_df.drop('Unnamed: 0', axis = 1, inplace = True)

In [41]:
words_df

,WORD,COUNT,DELETE
0,and,36926,0.0
1,of,24168,0.0
2,the,17842,0.0
3,to,15360,0.0
4,in,14512,0.0
...,...,...,...
134637,signs.nthrough,1,0.0
134638,sangnsongs,1,0.0
134639,marched,1,0.0
134640,peacefulndemonstration.,1,0.0


In [42]:
words_df.fillna(0, inplace = True)

In [43]:
words_df

,WORD,COUNT,DELETE
0,and,36926,0.0
1,of,24168,0.0
2,the,17842,0.0
3,to,15360,0.0
4,in,14512,0.0
...,...,...,...
134637,signs.nthrough,1,0.0
134638,sangnsongs,1,0.0
134639,marched,1,0.0
134640,peacefulndemonstration.,1,0.0


In [44]:
domain_stop_words_df = words_df[words_df.DELETE == 1]

In [45]:
domain_stop_words_df.drop('DELETE', axis = 1, inplace = True)

C:\Users\Hamna\AppData\Local\Temp\ipykernel_10592\3212713968.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  domain_stop_words_df.drop('DELETE', axis = 1, inplace = True)


In [46]:
domain_stop_words_df

,WORD,COUNT
12,management,2539
14,university,2325
17,business,2116
18,project,2061
28,development,1683
...,...,...
134195,parcel,1
134278,fractions,1
134473,substituting,1
134604,wonderful,1


In [47]:
data.index = range(0, len(data))

In [48]:
def remove_domain_stop_words(stop_words, df_edit):
    df_after = df_edit.copy()
    for i in range(len(df_after)):
        for word in stop_words:
            pattern = r'\b' + str(word) + r'\b'
            if isinstance(df_after.TEXT[i], str): # Check if the value is a string
                new_text = re.sub(pattern,'', df_after.TEXT[i])
                df_after.at[i, 'TEXT'] = new_text
    return df_after

In [49]:
df_after = remove_domain_stop_words(list(domain_stop_words_df.WORD), data)

In [50]:
# Replace 'xexx' with an empty string in the 'TEXT' column
df_after['TEXT'] = df_after['TEXT'].apply(lambda x: re.sub(r'xexx', '', x))

In [51]:
df_after.head(2)

,TEXT
0,programming languages pandas numpy scipy scikitlearn matplotlib javascriptjquery. learning regression svm nave bayes knn random forest decision trees boosting techniques cluster word embedding sentiment natural language processing dimensionality reduction topic modelling lda nmf pca neural nets. visualizations sqlserver cassandra hbase elasticsearch d.js dc.js plotly kibana matplotlib ggplot tableau. others regular expression angular logstash kafka flask git docker vision cv and understanding of learning. details assurance associatedata assurance associate ernst young llpskill details monthsjquery monthspython monthscompany details ernst young llpdescription fraud investigations and dispute assurancetechnology reviewtar review assists in accelerating the review process and run analytics and generate . core of a in developing automated review platform tool from scratch for assisting e discovery domain this tool implements predictive coding and topic modelling by automating in costs and time spent during the review. the end to end flow of the doing and for classification models predictive and mining of the present in text . worked on analyzing the outputs and precision monitoring for the entire tool. tar assists in predictive coding topic modelling from the evidence by following ey standards. the classifier models in order to identify red flags and fraudrelated issues.tools technologies scikitlearn tfidf wordvec docvec cosine similarity nave bayes lda nmf for topic modelling vader and text blob for sentiment . matplot lib tableau dashboard for multiple and analytic usa clientstext analytics motor vehicle review received feedback survey for past one . performed sentiment positive negative neutral and time on comments across all categories. created heat map of terms by survey category on frequency of words extracted positive and negative words across all the survey categories and plotted word cloud. created customized tableau dashboards for effective and visualizations.chatbot a user friendly chatbot for one of our products which handle simple questions about of operation reservation options and so on. this chat bot serves entire related questions. giving overview of tool via qa platform and also give recommendation responses so that user question to build of relevant answer. this too has intelligence to build the pipeline of questions as per user and asks the relevant recommended questions.tools technologies natural language processing nltk spacy topic modelling sentiment word embedding scikitlearn javascriptjquery sqlserverinformation governanceorganizations to make informed decisions about all of the they . the integrated portfolio synthesizes intelligence across unstructured sources and facilitates action to are best positioned to counter risk. scan from multiple sources of formats and parse different file formats extract meta push results for indexing elastic and created customized interactive dashboards kibana. preforming rot on the which give of which helps identify content that is either redundant outdated or trivial. preforming fulltext search on elastic with predefined methods which can tag as pii personally identifiable security numbers addresses etc. which frequently targeted during cyberattacks.tools technologies flask elastic kibanafraud analytic platformfraud analytics and investigative platform to review all red flag cases. fap is a fraud analytics and investigative platform with inbuilt case and suite of analytics for various erp . it can be used by to interrogate their for the anomalies which can be indicators of fraud by running advanced analyticstools technologies sqlserver jquery bootstrap node.js d.js dc.js
1,details may to may b.e uitrgpvdata scientistdata scientist matelabsskill details less than monthsstatsmodels monthsaws less than monthsmachine learning less than monthssklearn less than monthsscipy less than monthskeras less than monthscompany details matelabsdescription ml platform for professio

In [52]:
df_after.tail(2)

,TEXT
1255,bmajor conrad a. preedomn fairchild dr. suite hnusaf co n nconrad.preedomusafa.edunteaching experiencenassistant of director of curriculum us force co presentncoordinated over annually for members and over cadets curriculum for advancednmanagement provided classroom instruction to sections of cadets over semesters in multiple upperlevelncourses provided focused mentorship to more cadets.ndiamond star ta pilot us force co presentnperformed pilot duties in a singleengine diamond star da trainer aircraft provided instruction tonseasoned force pilots as well as us force cadets both in and out of the cockpit.ncessna tb pilot laughlin force base nperformed pilot duties at the us forces largest pilot base in forces primary jet trainernprovided both classroom instruction and flight instruction in all of flying to include traffic patternslandingsnaerobatic flight spin instrument flying formation flying and both day and night cross country navigation.nschweitzer pilot us force co nperformed pilot duties in the us force academys soar for all provided flightline and sailplanencockpit instruction to semesters of cadets with a focus on aerotow primary maneuvering traffic patterns and landing.nflying experiencen present flight pilot in diamond star da usaf tn n flight lead in mcdonnell douglas fe strike eaglen n in northrop atbn n flight pilot in cessna tbn n flight in northrop tan n flight pilot in schweitzer usaf tgnother aircraft flown include the slingsby ta firefly cessna tcd and the cessna .nusaf duty historyn present assistant of director of curriculum us force con ndirector of seventh force operationsplans a a osan base republic of korean ncommander of plans flight fe multiship flight lead seymour johnson force base ncn nfe student pilot seymour johnson force base ncn nexecutive officer th group tb pilot laughlin force base txn nus force pilot student laughlin force base txnformal educationnmba administration of pittsburgh pa gpa .nbs us force co gpa .nprofessional military educationnair command and nsquadron officer nratings and certificationsncertified flight federal aviation administration nsenior pilot united force ncommercial pilot federal aviation administration npilot united force nbasic parachutist united force nnxchonor and awardsnfield grade officer of the quarter division dept of us force co noutstanding academy award dept of us force co nfield grade officer of the quarter division dept of us force co nf. theodore helmer award of us force co ncompany grade officer of the quarter th support squadron seymour johnson force base nc nfe award th squadron seymour johnson force base nc ninstructor pilot of the th flying squadron laughlin force base ncompany grade officer of the quarter th flying squadron laughlin force base nbeta gamma sigma honor society katz of of pittsburgh pa ndistinguished united force co nomega rho honor society for united force co nair force awards and decorationsnmeritorious medalnair force commendation medalnair force achievement medalnair force outstanding unit awardnair force organizational awardncombat readiness medalnnational defense medalnglobal war on terrorism medalnkorean defense medalnaf longevity servicensmall arms expert marksmanship ribbon pistolnaf ribbon
1256,bstacy samplenn cellnnemail.comnnqualificationsnnflight timennflight ratingsnfaa atp airline transport pilotncfi rotorcrafthelicopterns s typeratingsnmedicalnfaa class nnexperiencensikorsky s s test pilotnjul presentnntotalnnnnpilot in commandnpilotnhelicopter multienginencross countryninstrument ifrhoodnactual weathernnnnnnnnnnightnnight vision gogglenoffshoreoverwaternexternal loadnemsnmountainnnnnnnnnnsikorsky aircraft corporation coatesville pans and s production test pilot maintenance test pilot engineeringntest flight civil and foreign military flight customerndemonstrations ifr vfr and overwater .nnbell ifr captainnjan jun nnelite aviation los angeles cansingle pilot bell ifr captain offairport landingnlogistics and a

In [53]:
df_after.sample(2)

,TEXT
895,bfinancial reporting resumennmack johnsonn north first streetnwinterset iowa nnobjectivento obtain a with a progressive organization that will utilize my skillsnabilities and in .nknowledge and skillsnxefxxbknowledge of gaap ifrs and sec and nxefxxbexceptional verbal written and organizational .nxefxxbexperience in preparing and filing quarterly and annual sec filings.nxefxxbstrong analytical .nxefxxbability to independently.nxefxxbstrong .ncomputer skillsnmicrosoft word excel and power point lotus lotus notes platinum etc.nacademic qualificationnbachelors in northern illinois ncertified public cpa nprofessional experiencenakamai technologies inc. iowannfinancial managernresponsibilitiesnxefxxbmanaged the reporting processes and procedures for capturingnmaintaining and reporting among provider agencies.nxefxxbestablished relationships with provider agencies rsn andninformation to assure understanding of and compliance with processesnprocedures and interfaces which support and timely report submissions.nxefxxbdeveloped and adapted tools and flows and securitynrequirements and to assure and timely datanextraction and report submission.nxefxxbprepared consolidated balance sheets profit and loss statements and flownstatements performed .nxefxxbworked with complex multicompany consolidations.neaton corporation carol stream illinoisnnnxcreporting managernresponsibilitiesnxefxxbcommunicated with advisors of the group to explain results of the groupnas reported in statements.nxefxxbmanagement of audit process people .nxefxxbperformed high of statements to identify issuesnand resolution of these issues.nxefxxbensured that all statutory filings are filed in a timely manner.
1169,bpnansncnannmichael archern n barranca avenm ca nsannannnnemailnnangnobjective to obtain a banking assistant in a growthoriented where i can utilize my administrativenenskills and acquire new abilities.ndnsummarynof skillsncnnonnxefxxbhugenexperience in bankingrelated compliance issuesnmnnpnxefxxbsolidnunderstanding of loan preparation review closing and nlnenxnxefxxbstrongncomputer including applicationsnsnxefxxbexceptionalnclient servicing along with excellent phone skillsnintnxefxxbgoodnproblem solving and decisionmaking skillsnunanxefxxbexcellent interpersonal written and verbal skillsntninwork experiencenonnnbankingnassistantnsnbanknleumi san cannaugust to till datenncnnxefxxbprovidedndirect to private banking .nonnonnrnxefxxbassistednin providing administrative support to private banking .ndninxefxxbprocessednaccount opening for both banking and investment .nnnanxefxxbmaintainednrecords and .ntninbanking assistantnnnjpmorgan chase san cangnmay to nsnanlnxefxxbhandlednclient transactions.nensnxefxxbresolvednroutine and complex inquiries and issues.nnnxefxxbprovided proactive support to which had existing relationships.nnsnanlnensnnxcxefxxbgathered and inputted related to lending.neducationnbachelors in accountingnduke durham nc


# Store The Cleaned Dataset Into Pickles

In [54]:
write_pickle('df_2_after_cleaning.pickle', df_after)

# Add SPACY_TEXT Colum

In [55]:
nlp = spacy.load('en_core_web_sm')
df_after['SPACY_TEXT'] = list(nlp.pipe(df_after.TEXT))

In [56]:
df_after.head(2)

,TEXT,SPACY_TEXT
0,programming languages pandas numpy scipy scikitlearn matplotlib javascriptjquery. learning regression svm nave bayes knn random forest decision trees boosting techniques cluster word embedding sentiment natural language processing dimensionality reduction topic modelling lda nmf pca neural nets. visualizations sqlserver cassandra hbase elasticsearch d.js dc.js plotly kibana matplotlib ggplot tableau. others regular expression angular logstash kafka flask git docker vision cv and understanding of learning. details assurance associatedata assurance associate ernst young llpskill details monthsjquery monthspython monthscompany details ernst young llpdescription fraud investigations and dispute assurancetechnology reviewtar review assists in accelerating the review process and run analytics and generate . core of a in developing automated review platform tool from scratch for assisting e discovery domain this tool implements predictive coding and topic modelling by automating in costs and time spent during the review. the end to end flow of the doing and for classification models predictive and mining of the present in text . worked on analyzing the outputs and precision monitoring for the entire tool. tar assists in predictive coding topic modelling from the evidence by following ey standards. the classifier models in order to identify red flags and fraudrelated issues.tools technologies scikitlearn tfidf wordvec docvec cosine similarity nave bayes lda nmf for topic modelling vader and text blob for sentiment . matplot lib tableau dashboard for multiple and analytic usa clientstext analytics motor vehicle review received feedback survey for past one . performed sentiment positive negative neutral and time on comments across all categories. created heat map of terms by survey category on frequency of words extracted positive and negative words across all the survey categories and plotted word cloud. created customized tableau dashboards for effective and visualizations.chatbot a user friendly chatbot for one of our products which handle simple questions about of operation reservation options and so on. this chat bot serves entire related questions. giving overview of tool via qa platform and also give recommendation responses so that user question to build of relevant answer. this too has intelligence to build the pipeline of questions as per user and asks the relevant recommended questions.tools technologies natural language processing nltk spacy topic modelling sentiment word embedding scikitlearn javascriptjquery sqlserverinformation governanceorganizations to make informed decisions about all of the they . the integrated portfolio synthesizes intelligence across unstructured sources and facilitates action to are best positioned to counter risk. scan from multiple sources of formats and parse different file formats extract meta push results for indexing elastic and created customized interactive dashboards kibana. preforming rot on the which give of which helps identify content that is either redundant outdated or trivial. preforming fulltext search on elastic with predefined methods which can tag as pii personally identifiable security numbers addresses etc. which frequently targeted during cyberattacks.tools technologies flask elastic kibanafraud analytic platformfraud analytics and investigative platform to review all red flag cases. fap is a fraud analytics and investigative platform with inbuilt case and suite of analytics for various erp . it can be used by to interrogate their for the anomalies which can be indicators of fraud by running advanced analyticstools technologies sqlserver jquery bootstrap node.js d.js dc.js,"( , programming, languages, , pandas, numpy, scipy, scikitlearn, matplotlib, , javascriptjquery, ., , learning, regression, svm, nave, bayes, knn, random, forest, decision, trees, boosting, techniques, cluster, , word, embedding, sentiment, , natural, language, processing, dimensionalit

In [57]:
df_after.tail(2)

,TEXT,SPACY_TEXT
1255,bmajor conrad a. preedomn fairchild dr. suite hnusaf co n nconrad.preedomusafa.edunteaching experiencenassistant of director of curriculum us force co presentncoordinated over annually for members and over cadets curriculum for advancednmanagement provided classroom instruction to sections of cadets over semesters in multiple upperlevelncourses provided focused mentorship to more cadets.ndiamond star ta pilot us force co presentnperformed pilot duties in a singleengine diamond star da trainer aircraft provided instruction tonseasoned force pilots as well as us force cadets both in and out of the cockpit.ncessna tb pilot laughlin force base nperformed pilot duties at the us forces largest pilot base in forces primary jet trainernprovided both classroom instruction and flight instruction in all of flying to include traffic patternslandingsnaerobatic flight spin instrument flying formation flying and both day and night cross country navigation.nschweitzer pilot us force co nperformed pilot duties in the us force academys soar for all provided flightline and sailplanencockpit instruction to semesters of cadets with a focus on aerotow primary maneuvering traffic patterns and landing.nflying experiencen present flight pilot in diamond star da usaf tn n flight lead in mcdonnell douglas fe strike eaglen n in northrop atbn n flight pilot in cessna tbn n flight in northrop tan n flight pilot in schweitzer usaf tgnother aircraft flown include the slingsby ta firefly cessna tcd and the cessna .nusaf duty historyn present assistant of director of curriculum us force con ndirector of seventh force operationsplans a a osan base republic of korean ncommander of plans flight fe multiship flight lead seymour johnson force base ncn nfe student pilot seymour johnson force base ncn nexecutive officer th group tb pilot laughlin force base txn nus force pilot student laughlin force base txnformal educationnmba administration of pittsburgh pa gpa .nbs us force co gpa .nprofessional military educationnair command and nsquadron officer nratings and certificationsncertified flight federal aviation administration nsenior pilot united force ncommercial pilot federal aviation administration npilot united force nbasic parachutist united force nnxchonor and awardsnfield grade officer of the quarter division dept of us force co noutstanding academy award dept of us force co nfield grade officer of the quarter division dept of us force co nf. theodore helmer award of us force co ncompany grade officer of the quarter th support squadron seymour johnson force base nc nfe award th squadron seymour johnson force base nc ninstructor pilot of the th flying squadron laughlin force base ncompany grade officer of the quarter th flying squadron laughlin force base nbeta gamma sigma honor society katz of of pittsburgh pa ndistinguished united force co nomega rho honor society for united force co nair force awards and decorationsnmeritorious medalnair force commendation medalnair force achievement medalnair force outstanding unit awardnair force organizational awardncombat readiness medalnnational defense medalnglobal war on terrorism medalnkorean defense medalnaf longevity servicensmall arms expert marksmanship ribbon pistolnaf ribbon,"(bmajor, conrad, a., preedomn, fairchild, dr, ., suite, hnusaf, co, n, nconrad.preedomusafa.edunteaching, experiencenassistant, , of, , director, of, curriculum, us, , force, co, presentncoordinated, over, , annually, for, , members, and, over, , cadets, , curriculum, for, advancednmanagement, , provided, classroom, instruction, to, , sections, of, cadets, over, , semesters, in, multiple, upperlevelncourses, provided, focused, , mentorship, to, more, , cadets.ndiamond, star, ta, , pilot, us, , force, co, presentnperformed, , pilot, duties, in, a, singleengine, diamond, star, da, trainer, aircraft, provided, instruction, tonseasoned, , force, pilots, as, well, as, us, , force, cadets, both, in, and, out, of, the, ...)"

In [58]:
df_after.sample(2)

,TEXT,SPACY_TEXT
415,bharold hotellingnuniversity of north carolinandepartment of statisticsnsmith buildingnchapel hill nc nnphonenfaxnemailnhomepagenn n nhotellingunc.edunhttpwww.stator.unc.edunnpersonalnborn on .nunited citizen.nneducationnb.s. journalism .nm.a. .nph.d. princeton .nnemploymentnstanford .ncolumbia .nuniversity of north carolina .nnpublicationsnjournal articlesna mathematical of depreciation of the statistical association n.ndifferential equations subject to error of the statistical association.napplications of the of error to the interpretation of trends with h. ofnthe statistical association.nnproceedingsna generalized ttest and measure of multivariate dispersion proc. second berkeley symposium ofnmathematical statistics and probability .nnlast updated nnhttpjblevins.orgprojectscvtemplate,"(bharold, hotellingnuniversity, of, north, carolinandepartment, of, statisticsnsmith, buildingnchapel, hill, nc, nnphonenfaxnemailnhomepagenn, n, nhotellingunc.edunhttpwww.stator.unc.edunnpersonalnborn, on, , .nunited, , citizen.nneducationnb.s, ., journalism, , .nm.a, ., , .nph.d, ., , princeton, , .nnemploymentnstanford, , .ncolumbia, , .nuniversity, of, north, carolina, .nnpublicationsnjournal, articlesna, , mathematical, , of, depreciation, , of, the, , statistical, association, n.ndifferential, equations, subject, to, error, , of, the, , statistical, association.napplications, of, the, , of, error, to, the, interpretation, of, trends, with, h., , ofnthe, , statistical, association.nnproceedingsna, generalized, ttest, and, measure, of, multivariate, dispersion, proc, ., second, berkeley, symposium, ofnmathematical, statistics, and, probability, .nnlast, updated, , nnhttpjblevins.orgprojectscvtemplate)"
980,bjesse kendalln elm street miami fl jkendallnotmail.comnnindustrial engineerncompetent and resultsoriented with extensive innindustrial . proficient in word excelnpowerpoint sap . consistently promoted into ofnincreased . fluent in and . black belt technicalncertification.nxefxxb systemsnxefxxb jit kanbannxefxxb managementnnxefxxb lean manufacturingnxefxxb improvement techniquesnxefxxb strategic and tacticalnplanningnnxefxxb leadershipnxefxxb process developmentnxefxxb budget managementnnengineering excellencenindustrial engineer abc enterprises miami fl xx xxncompleted and participated in new plant . coordinated and interfacednbetween the and plant personnel in layout installationnreview process validation and results for the scrap linesncapability and productivity improvements. oversaw delphi manufacturingnsystems lean techniques and to assure better utilization.nanalyzed productivity trends and manufacturing metrics. developedndocumentation and action plans for pfd pfmea and pcp . supervisednstaff of employees.nxefxxbsaved per by parameters and reducing scrap innthe solder equipment.nxefxxbcontrolled costs at two plants saving . and . million.ndivisional industrial engineer bcd vehicles miami fl xx xxndirected all activities for three stamping and component plants. trackednhourly and salary manpower levels at each plant. analyzed validity of additionalnstaffing requests on yearend targets. communicated with uawnleadership productivity shift and volume changes.nnxcxefxxbdeveloped estimates for future stamping on new car programsnincluding die configuration equipment capabilities and manpowernrequirements.nxefxxbselected to travel to other plants to improve processes andnprofitability.nxefxxbexceeded manpower reduction targets for entire division for threenconsecutive years.nxefxxbsaved . million by implementing direct ship to austria.nxefxxbreduced the of outside consolidation by .nindustrial engineer cde plant miami fl xx xxnoversaw budget and capacity for the plant. evaluatednlabor tooling and physical site costs with the of identifyingnopportunities for improvement. negotiations with the union and nonunion force. ensured compliance with standards.nxefxxbapplied best practice manufacturing and logistics managemen

In [59]:
# Delete any Entities 
for i in range(0, len(df_after)):
    ents = df_after.SPACY_TEXT[i]
    j = 0
    for ent in ents.ents:
        pattern = r'\b' + ent.text + r'\b'
        new_text = re.sub(pattern,' ', df_after.TEXT[j])
        df_after.TEXT[j] = new_text
        j += 1

In [60]:
df_after.TEXT = df_after.TEXT.apply(lambda x: x.lower())

In [61]:
df_after.head(2)

,TEXT,SPACY_TEXT
0,programming languages pandas numpy scipy scikitlearn matplotlib javascriptjquery. learning regression svm nave bayes knn random forest decision trees boosting techniques cluster word embedding sentiment natural language processing dimensionality reduction topic modelling lda nmf pca neural nets. visualizations sqlserver cassandra hbase elasticsearch js .js plotly matplotlib ggplot tableau. others regular expression angular logstash kafka flask git docker vision cv and understanding of learning. details assurance associatedata assurance associate ernst young llpskill details monthsjquery monthscompany details ernst young llpdescription fraud investigations and dispute assurancetechnology reviewtar review assists in accelerating the review process and run analytics and generate . core of a in developing automated review platform tool from scratch for assisting e discovery domain this tool implements predictive coding and topic modelling by automating in costs and time spent during the review. the end to end flow of the doing and for classification models predictive and mining of the present in text . worked on analyzing the outputs and precision monitoring for the entire tool. tar assists in predictive coding topic modelling from the evidence by following ey standards. the classifier models in order to identify red flags and fraudrelated issues.tools technologies scikitlearn tfidf wordvec docvec cosine similarity nave bayes lda nmf for topic modelling vader and text blob for sentiment . matplot lib tableau dashboard for multiple and analytic usa clientstext analytics motor vehicle review received feedback survey for past . performed sentiment positive negative neutral and time on comments across all categories. created heat map of terms by survey category on frequency of words extracted positive and negative words across all the survey categories and plotted word cloud. created customized tableau dashboards for effective and visualizations.chatbot a user friendly chatbot for of our products which handle simple questions about of operation reservation options and so on. this chat bot serves entire related questions. giving overview of tool via qa platform and also give recommendation responses so that user question to build of relevant answer. this too has intelligence to build the pipeline of questions as per user and asks the relevant recommended questions.tools technologies natural language processing nltk spacy topic modelling sentiment word embedding scikitlearn javascriptjquery sqlserverinformation governanceorganizations to make informed decisions about all of the they . the integrated portfolio synthesizes intelligence across unstructured sources and facilitates action to are positioned to counter risk. scan from multiple sources of formats and parse different file formats extract meta push results for indexing elastic and created customized interactive dashboards . preforming rot on the which give of which helps identify content that is either redundant outdated or trivial. preforming fulltext search on elastic with predefined methods which can tag as pii personally identifiable security numbers addresses etc. which frequently targeted during cyberattacks.tools technologies flask elastic kibanafraud analytic platformfraud analytics and investigative platform to review all red flag cases. fap is a fraud analytics and investigative platform with inbuilt case and suite of analytics for various erp . it can be used by to interrogate their for the anomalies which can be indicators of fraud by running advanced analyticstools technologies sqlserver jquery bootstrap node.js js .js,"( , programming, languages, , pandas, numpy, scipy, scikitlearn, matplotlib, , javascriptjquery, ., , learning, regression, svm, nave, bayes, knn, random, forest, decision, trees, boosting, techniques, cluster, , word, embedding, sentiment, , natural, language, processing, dimensionality, reduction, topic, modelling, lda, nmf, pca, 

# Topic Modeling and Removing Domain Specific Stop Words Iterations

In [62]:
vectorizer = CountVectorizer(max_features=20000,
                             stop_words='english', token_pattern="\\b[a-z][a-z]+\\b",
                             binary=True)

doc_word = vectorizer.fit_transform(df_after.TEXT)
words = list(np.asarray(vectorizer.get_feature_names_out()))

In [63]:
topic_model = ct.Corex(n_hidden=6, words=words, seed=1)
topic_model.fit(doc_word, words=words, docs=df_after.TEXT)

In [64]:
# Print all topics from the CorEx topic model
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))
topics

0: society,national,group,review,committee,association,director,public,world,human
1: including,strategic,new,major,programs,implemented,million,plan,goals,process
2: testing,user,developer,worked,users,test,used,visual,programming,languages
3: dayjob,readnxcxa,infodayjob,copyright,welcome,nnxccopyright,permission,download,jobseekers,template
4: availability,performed,etl,tr,structures,backups,dba,environments,estimating,cobol
5: monthscompany,details,pvt,ltddescription,pune,like,ltdskill,description,various,mvc


[[('society', 0.112053327798441, 1.0),
  ('national', 0.09286578860473911, 1.0),
  ('group', 0.0785393136471408, 1.0),
  ('review', 0.07715376832550404, 1.0),
  ('committee', 0.07638962857908914, 1.0),
  ('association', 0.07247955614749631, 1.0),
  ('director', 0.06826610797301584, 1.0),
  ('public', 0.0633729636994509, 1.0),
  ('world', 0.06308241308698052, 1.0),
  ('human', 0.062253854342538335, 1.0)],
 [('including', 0.16522728331031067, 1.0),
  ('strategic', 0.12671214665372843, 1.0),
  ('new', 0.08858357524627354, 1.0),
  ('major', 0.08506363234563445, 1.0),
  ('programs', 0.08355439036906893, 1.0),
  ('implemented', 0.08111530417729858, 1.0),
  ('million', 0.07999576066875054, 1.0),
  ('plan', 0.07342757606911117, 1.0),
  ('goals', 0.07303204389304548, 1.0),
  ('process', 0.07103649926780294, 1.0)],
 [('testing', 0.09142610571161873, 1.0),
  ('user', 0.087177539788353, 1.0),
  ('developer', 0.08344566549291478, 1.0),
  ('worked', 0.07553597991266393, 1.0),
  ('users', 0.072885538

In [65]:
tfidf = TfidfVectorizer(stop_words='english')
doc_words = tfidf.fit_transform(df_after.TEXT)
tfidf_df = pd.DataFrame(doc_words.toarray(),columns=tfidf.get_feature_names())

C:\Users\Hamna\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [66]:
topic_nums = list(np.arange(3, 11, 1))
for num in topic_nums:
    nmf = NMF(num)
    nmf.fit_transform(doc_words)
    display_topics(nmf, tfidf.get_feature_names_out(), 100)
    print("------------------------------------------")

C:\Users\Hamna\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
na, new, including, legal, student, support, process, high, activities, employees, million, years, programs, ability, york, customers, market, various, human, time, administration, public, assistant, procedures, include, com, annual, products, account, association, group, issues, ca, revenue, strong, key, plans, related, strategic, community, nxefxxa, implemented, cv, organization, budget, banking, develop, needs, provided, national, processes, providing, care, managing, relations, society, policies, create, building, available, excel, tax, designed, safety, major, executive, compliance, daily, effective, implementation, review, teams, strategies, help, production, administrative, goals, make, plan, committee, materials, risk, site, special, developing, academic, gpa, word, request, created, letter, best, industry, events, recruitment, relationships, equipment, internal, manage, consulting

Topic  1
nxefxxb, responsible, customers, generating, analyzing, tasks, excellent, neg

C:\Users\Hamna\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
na, new, including, student, process, million, employees, activities, support, york, programs, high, years, legal, human, various, market, public, administration, annual, include, association, assistant, time, ca, group, ability, products, procedures, nxefxxa, community, implemented, related, issues, organization, revenue, customers, strategic, banking, provided, society, national, designed, strong, tax, relations, account, major, policies, budget, plans, develop, care, excel, key, committee, needs, processes, review, gpa, compliance, implementation, created, risk, executive, building, production, strategies, special, word, academic, teams, plan, goals, providing, safety, daily, consulting, teacher, prepared, summer, st, managing, personnel, letter, effective, materials, internal, order, board, recruitment, make, industry, administrative, course, equipment, san, events, division, best

Topic  1
nxefxxb, responsible, generating, customers, analyzing, tasks, excellent, negotiat

C:\Users\Hamna\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


...


Topic  0
new, process, employees, including, million, support, human, various, banking, market, administration, processes, programs, procedures, tax, activities, policies, compliance, products, strategic, customers, years, risk, revenue, account, recruitment, issues, internal, plans, implementation, ability, budget, develop, time, executive, teams, managing, key, credit, manage, implemented, plan, strategies, test, implementing, capital, related, york, provided, personnel, annual, consulting, strategy, benefits, delivery, group, effective, organization, providing, goals, companys, global, lead, user, excel, report, relations, responsibilities, branch, production, vendor, relationships, sap, strong, organizational, division, testing, developing, multiple, retail, daily, inventory, manufacturing, maintain, contract, major, timely, nxefxxa, loan, application, identify, audit, existing, functional, ensured, flow, opportunities, compensation, practices, areas

Topic  1
nxefxxb, responsible

C:\Users\Hamna\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
test, testing, application, user, automation, net, windows, process, developer, functional, users, used, tools, technologies, pvt, cases, support, visual, involved, like, various, worked, framework, responsibilities, asp, environment, tool, hadoop, designed, programming, ltddescription, ui, created, hive, implementation, jquery, report, sap, role, document, creating, teams, platform, agile, manual, architecture, uat, shell, different, studio, issues, gathering, processes, configuration, mobile, mysql, engineer, end, flash, monitoring, time, production, regression, selenium, designing, core, site, security, lead, xp, script, custom, maintenance, modules, defect, creation, backup, tech, access, release, languages, scripting, enterprise, qa, etl, internal, ux, developing, develop, tests, create, files, automated, scenarios, change, specifications, multiple, manage, performed, mvc

Topic  1
nxefxxb, responsible, customers, generating, analyzing, excellent, tasks, negotiations, re

C:\Users\Hamna\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
test, testing, application, user, automation, net, windows, process, functional, developer, users, used, tools, cases, pvt, technologies, involved, support, visual, like, framework, various, worked, responsibilities, asp, environment, hadoop, tool, ltddescription, ui, programming, designed, hive, implementation, created, report, sap, role, jquery, manual, teams, uat, agile, creating, document, platform, architecture, shell, issues, gathering, different, studio, processes, configuration, selenium, regression, mobile, mysql, monitoring, end, flash, core, engineer, production, security, designing, defect, site, time, script, xp, lead, creation, modules, release, custom, backup, tech, maintenance, access, qa, scripting, enterprise, etl, languages, ux, internal, change, scenarios, case, automated, tests, documents, developing, specifications, develop, files, multiple, create, uml

Topic  1
nxefxxb, responsible, customers, generating, analyzing, excellent, tasks, negotiations, resp

In [67]:
stop_words = ['format', 'want', 'contact', 'adres', 'relevant', 'avoid', 'aplying', 
              'demonstrate', 'examples', 'employer', 'refer', 'reference', 'phone', 
              'start', 'colege', 'posible', 'sure', 'important', 'use', 'number', 'verbs', 
              'style', 'cover', 'include', 'action', 'clas', 'type', 'leter', 'kep', 
              'separate', 'font', 'draft', 'person', 'aply', 'acomplishments', 'thre', 'speak', 
              'way', 'read', 'ned', 'degre', 'employers', 'volunter', 'begin', 'past',
              'language', 'aditional', 'submit', 'guide', 'candidate',
              'clases', 'highlight', 'make', 'copy', 'oportunity',
              'think', 'aid', 'title', 'lok', 'location', 'organize', 'serve', 'print', 
              'supervise', 'atention', 'contribute', 'expect', 'seking', 'chronological',
              'ask', 'reverse', 'compile', 'white', 'explain', 'coursework',  'easy',
              'note', 'honors', 'brief', 'emphasize', 'erors', 'short', 'construct', 'writen', 
              'author', 'place', 'abroad', 'consider', 'gain', 'choose', 'advise', 
              'know', 'speling', 'internship', 'recent','asociation', 'research', 
              'curiculum', 'program', 'public', 'council', 
              'publications', 'comunity',  'awards',  
              'science', 'practice', 'asociate', 
              'presentations', 'academy', 'evaluation',  'social', 'chair',  'foundation', 
              'studies', 'activities',  'asesment', 'special', 
              'workshop', 'study', 'grant', 'human', 'work', 'world', 
              'felow', 'year', 'honors', 
              'undergraduate', 'life',  'chapter', 
              'seminar', 'non', 'private', 'policy', 'cordinator', 'curent',   
              'future', 'clas',  'conference', 
              'united', 'county', 'home', 'family', 'pres', 'symposium', 'forum', 'john',
              'isues', 'bok', 'case', 'series', 'scholarship', 'history', 
              'graduate', 'editor', 'outstanding', 'suport', 'learning', 'regional',
              'administrative',  'young', 'second', 'thre', 'court', 'initiative'
              'available', 'request', 'personal', 'english', 'references', 'template',
              'example', 'loking', 'info', 'question', 'read', 'qualifications', 'welcome', 
              'prior', 'websites',   'permision', 'download', 'distribute',
              'link', 'dayjob', 'copyright', 'able', 'ccopyright', 'areas', 'duties', 'key', 
              'expertise',  'competencies', 'hons', 'coventry',  'sumary', 
              'details',  'highly', 'nationality',  'jobsekers', 'profesional',
              'driving', 'birmingham', 'license', 'british', 'environment', 'central',
              'work', 'yes', 'suitable', 'nuneaton',  
              'senior', 'working', 'history',  
              'deliver',  'ambitious',  'having',
              'creative', 'providing', 'atention', 'people', 'coleagues', 'carer', 'drive',
              'extensive', 'particular', 'road', 'pose', 'contact', 'sekers', 'run',
              'dob', 'right', 'individual', 'prove', 
              'confident', 'diploma', 'asisting', 'motivate', 'big' ,'interpreted', 
              'arbitrated', 'shaped', 'invented', 'delegated', 'explained', 'surveyed', 
              'wrote', 'inspected',  'checked', 'calculated', 'compared', 
              'stimulated', 'repaired', 'weight', 'fabricated', 'scheduled', 'specialized', 
              'programed', 'contracted', 'adresed', 'educated', 'advertised', 'atained', 
              'modified', 'reinforced', 'examined', 'synthesized', 'acomplished', 'estimated',
              'criteria', 'solved', 'operated', 'directed', 'asembled',
              'influenced' , 'executed', 'persuaded', 'verified', 
              'adapted', 'computed', 'determined', 'fod', 'john', 'aranged', 'entertained', 'established', 
              'instituted', 'revised', 'separate', 'fel', 'furnished', 'originated', 'body',
              'status', 'conceived', 'aplicants', 'published', 'thank', 'unified', 'facilitated', 
              'guidelines', 'transmited', 'familiarized', 'say', 'wish', 'demonstrated', 
              'hired', 'discovered', 'indicate', 'instaled', 'anticipated', 'streamlined', 
              'projected', 'formulated', 'age', 'researched', 'underline', 'cover', 'asesed',
              'gathered', 'employer', 'proposed', 'sure', 'adreses', 'qualified', 'expanded',
              'detected', 'pionered', 'right', 'fashioned', 'clarified', 'indian', 'hindi', 'maried', 'male', 'xcxasoftwaretr', 'xcxafromtr', 'xcxaglobaltr', 
              'xcxaamericatr', 'xcxafranciscotr', 'xcxamanagedtr', 'xcxacollectionxcxadtr', 'xcxaantoniotr', 'xcxamanagementtr', 'xcxaincludingtr', 'xcxankeytr', 'xcxaastr', 'xcxamagazinetr', 'xcxafashiontr', 'xcxandentaltr', 'comtr', 'xcxacontractstr',
              'xcxacreatedtr', 'xcxamanagertr', 'xcxasheltontr', 'xcxastylisttr',
              'xcxamagazinexcxadtr', 'xcxahairt', 'xcxabrandtr', 'xcxarestorativetr', 'xcxatwotr', 'andtr', 'xcxageneraltr', 'xcxaexperiencetr', 'xcxatr', 'xcxantr', 'tr', 'xcxaandtr', 'xcxa', 'xcxaoftr', 'xcxaintr', 'xcxanntr', 'xcxatotr', 'xcxathetr', 'xcxana', 'xcxaxcxadtr', 'xcxaafortr', 'xcxamarketingtr', 'xcxatxtr', 'xcxabytr', 'xcxawithtr', 'xcxasalestr', 'xcxaaustintr', 'xcxabusinesstr', 'xcxaarttr', 'xcxanewtr', 'xcxaattr', 'xcxaatr', 'xcxacatr', 'xcxapracticetr', 'xcxagallerytr', 'xcxanparttr', 'xcxasantr', 'xcxadevelopmenttr', 'xcxaincreasedtr', 'xcxadesignertr', 'xcxanxcxadtr', 'xcxauniversitytr', 'xcxaontr', 'xcxanftr', 'xcxadentistrytr', 'xcxadentaltr', 'xcxamarkettr', 'xcxathtr', 'xcxaproducttr', 'xcxatexastr', 'xcxamediatr', 'xcxaaesthetictr', 'xcxastrategytr', 'xcxalatintr', 'xcxamtr', 'xcxarevenuetr', 'xcxacontracttr', 'xcxainternationaltr', 'xcxasoftwaretr', 'xcxafromtr', 'xcxaglobaltr', 'xcxaamericatr', 'xcxafranciscotr', 'xcxamanagedtr', 'xcxacollectionxcxadtr', 'xcxaantoniotr', 'xcxamanagementtr', 'xcxaincludingtr', 'xcxankeytr', 'xcxaastr', 'xcxamagazinetr', 'xcxafashiontr', 'xcxandentaltr', 'comtr', 'xcxacontractstr', 'xcxacreatedtr', 'xcxamanagertr',
              'xcxasheltontr', 'xcxastylisttr', 'xcxamagazinexcxadtr', 'xcxahairtr', 'xcxabrandtr', 'xcxarestorativetr', 'xcxatwotr']

In [68]:
df_after_2 = remove_domain_stop_words(stop_words = stop_words, df_edit = df_after)

In [69]:
df_after_2

TEXT                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      SPACY_TEXT
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    programming languages  pandas numpy scipy scikitlearn matplotlib   javascriptjquery.    regression svm nave bayes knn random forest decision trees boosting techniques cluster  word embedding sentiment  natural  processing dimensionality reduction topic modelling lda nmf pca  neural nets.   visualizations sqlserver cassandra hbase elasticsearch  js  .js plotly   matplotlib ggplot tableau.  others regular expression   angular  logstash kafka  flask git docker  vision   cv and understanding of  .    assurance associatedata  assurance associate  ernst   llpskill      monthsjquery         monthscompany    ernst   llpdescription  fraud investigations and dispute    assurancetechnology  reviewtar   review assists in accelerating the review process and  analytics and generate . core  of a   in developing automated review platform tool from scratch for assisting e discovery domain this tool implements predictive coding and topic modelling by automating   in   costs and time spent during the  review.  the end to end flow of the  doing  and  for classification models predictive  and mining of the  present in text . worked on analyzing the outputs and precision monitoring for the entire tool. tar assists in predictive coding topic modelling from the evidence by following ey standards.  the classifier models in order to identify red flags and fraudrelated issues.tools  technologies  scikitlearn tfidf wordvec docvec cosine similarity nave bayes lda nmf for topic modelling vader and text blob for sentiment . matplot lib tableau dashboard for multiple   and analytic  usa clientstext analytics  motor vehicle  review   received  feedback survey  for    . performed sentiment positive negative  neutral and time   on  comments across all  categories. created heat map of terms by survey category  on frequency of words  extracted positive and negative words across all the survey categories and plotted word cloud. created customized

In [70]:
tfidf2 = TfidfVectorizer(stop_words='english')
doc_words2 = tfidf2.fit_transform(df_after_2.TEXT)
tfidf_df2 = pd.DataFrame(doc_words2.toarray(),columns=tfidf2.get_feature_names_out())

In [71]:
topic_nums = list(np.arange(10, 20, 1))
for num in topic_nums:
    nmf = NMF(num)
    nmf.fit_transform(doc_words2)
    display_topics(nmf, tfidf2.get_feature_names_out(), 100)
    print("------------------------------------------")

C:\Users\Hamna\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
new, employees, million, process, including, banking, tax, market, administration, programs, account, revenue, strategic, policies, support, compliance, procedures, various, products, customers, risk, years, budget, executive, recruitment, processes, ability, credit, annual, internal, capital, strategies, plans, managing, issues, develop, branch, york, goals, personnel, time, implementation, companys, benefits, manage, relations, implemented, teams, organization, relationships, implementing, effective, division, related, provided, excel, consulting, retail, plan, organizational, strategy, strong, global, loan, opportunities, transactions, ensured, audit, major, group, statements, timely, vendor, initiatives, daily, compensation, industry, toronto, increase, investment, delivery, inventory, identify, markets, costs, savings, contract, ca, direct, practices, profitability, satisfaction, companies, developing, prepared, lead, maintain, payable, sap, cityland

Topic  1
nxefxxb, r

C:\Users\Hamna\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
new, million, employees, process, including, market, strategic, programs, revenue, recruitment, administration, executive, account, processes, policies, strategy, support, budget, benefits, risk, products, consulting, division, teams, relations, implementation, personnel, strategies, annual, companys, develop, global, goals, implemented, years, compensation, major, plans, nxefxxa, managing, york, compliance, relationships, contract, plan, contracts, costs, vendor, implementing, increase, organizational, lead, manage, nxefxx, direct, initiatives, time, opportunities, organization, profitability, increased, led, markets, building, practices, identify, manufacturing, savings, director, successful, improvement, brand, developing, leading, president, line, issues, portfolio, salary, responsible, region, delivery, retention, resource, group, toronto, internal, procedures, companies, communications, various, identified, advertising, campaigns, related, safety, created, territory, sa

C:\Users\Hamna\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


...


Topic  0
new, million, employees, including, process, market, strategic, revenue, programs, executive, administration, account, recruitment, processes, budget, support, risk, products, benefits, strategy, division, policies, annual, teams, strategies, consulting, relations, goals, global, companys, implementation, develop, implemented, personnel, plans, york, managing, major, years, relationships, compensation, plan, contract, contracts, increase, costs, compliance, organizational, direct, implementing, manage, vendor, initiatives, nxefxx, lead, opportunities, time, profitability, increased, markets, led, building, manufacturing, identify, organization, improvement, director, savings, brand, successful, practices, leading, developing, president, delivery, line, issues, retention, portfolio, territory, campaigns, nxcx, region, toronto, competitive, advertising, satisfaction, production, government, industry, achieve, identified, communications, procedures, los, pmo, related, latin, gro

C:\Users\Hamna\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
new, million, market, revenue, account, strategic, products, york, annual, executive, strategy, global, teams, strategies, relationships, brand, markets, including, campaigns, process, years, increase, profitability, risk, division, consulting, programs, major, advertising, territory, direct, increased, nxcx, budget, director, companys, goals, led, latin, region, implementation, lead, customers, processes, support, initiatives, national, plans, opportunities, share, content, contracts, largest, consumer, leading, digital, retail, industry, profit, president, competitive, implemented, contract, costs, achieved, months, nxcxbe, revenues, communications, portfolio, building, plan, identify, capital, managing, successful, channel, investment, relationship, responsible, companies, dealer, fortune, award, manage, ny, retention, organization, annually, pmo, delivery, relations, selling, multiple, develop, real, enterprise, trade, compliance, billion

Topic  1
nxefxxb, responsible, c

C:\Users\Hamna\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
million, new, market, revenue, account, strategic, products, annual, executive, york, strategy, global, teams, strategies, brand, relationships, process, markets, including, campaigns, risk, profitability, years, increase, division, consulting, major, direct, advertising, territory, programs, budget, increased, director, led, companys, implementation, goals, latin, region, processes, lead, nxcx, initiatives, support, national, opportunities, content, share, digital, contracts, profit, plans, leading, largest, president, implemented, competitive, industry, nxcxbe, consumer, contract, costs, retail, achieved, revenues, months, communications, portfolio, capital, investment, plan, customers, identify, managing, successful, channel, building, responsible, dealer, pmo, fortune, real, relationship, manage, companies, award, organization, annually, multiple, delivery, enterprise, retention, develop, compliance, relations, billion, crossfunctional, startup, ny

Topic  1
nxefxxb, resp

# Second Itration of Removing Domain Specific Word

In [72]:
stop_words_2 = ['resume', 'list', 'use', 'position',
                'job', 'present', 'include', 'special',
                'carer', 'adres', 'relevant', 'sumer', 'leter', 'fal',
                'state', 'parent','volunter', 'cover', 'wil',
                'section', 'comunity', 'member', 'objective','intern', 
                'society', 'title', 'make', 'clas', 'center', 'sample', 'august',
                'employer', 'internship', 'monash', 'page', 'graduate', 'profesional', 'asisted', 'language',
                'campus', 'honors', 'help', 'city', 'public', 'word', 'asociation', 'year', 'bachelor',
                'staf', 'contact','format', 'events', 'chicago', 'specific', 'date', 'social', 'nebraska', 
                'point', 'neds','project', 'use','team', 'end', 'create','proces','document','role',
                'experience', 'case', 'requirement', 'description', 'report', 'worked', 'used',
                'object', 'like', 'lead', 'source', 'custom', 'provide', 'enterprise', 'tol', 'hibernate',
                'new',  'team', 'representative', 'store', 'teritory', 
                'year','brand', 'experience', 'train', 'key', 'city', 'goals', 'profesional', 'years',
                'generate', 'present', 'regional', 'anual', 'quota', 'executive', 'representatives', 'lead',
                'achieved', 'build', 'skils', 'client', 'rep', 'oportunities', 'achievements', 
                'york', 'latin', 'region', 'achieve', 'texas', 'entries', 'knowledge', 
                'asociate', 'direct', 'industry', 'profitability', 'position', 'promotion', 'plan', 'consistently',
                'distribution', 'national', 'university', 'online', 'leadership', 'advertise',
                'kansas', 'profit', 'major', 'global', 'responsible', 'internet', 'bachelor', 'international',
                'statements','prepared', 'prepare', 'corporate', 'ledger', 'capital', 'company',
                'reconciliation', 'skils', 'experience','receivable','flow', 
                'internal', 'ability', 'preparation', 'entry', 'principles', 'parkland', 'general', 'project',
                'proces', 'plan', 'new', 'anual', 'quarterly','resume', 'cityland', 'review',
                'chicago', 'team', 'performed', 'work', 'cpa',
                'profesional', 'chartered', 'level', 'bachelor', 'fitzwiliam', 'email', 
                'strong', 'main', 'qualifications', 'performance', 'record', 'stret', 'senior', 'university'
                'exprience', 'details', 'months', 'les', 'description', 'maharashtra', 'year', 'company', 'pune',
                'january', 'monthscompany', 'mumbai', 'pvt', 'nagpur','india', 'june', 'universityhrskil', 
                'work', 'diploma', 'working', 'hibernate', 'worked', 'nashik', 'god', 'telangana', 'skils', 
                'requirement', 'learn', 'django', 'cricket', 'phule', 'savitribai', 'knowledge',
                'board', 'skil', 'amravati', 'secondary', 'university', 'role', 'servlet', 
                'institute', 'duration', 'title', 'solution', 'pradesh',
                'days', 'like', 'angular', 'job', 'solapur',
                'dayjob', 'personal', 'birmingham', 'template', 'coventry', 'able', 'use', 'info', 
                'available', 'work', 'skils', 'download', 'ability', 'distribute',
                'com', 'ccopyright', 'fod', 'copyright', 'make', 'permision', 'welcome', 
                'question', 'loking','prior', 'read', 'example', 'hons', 'cvs', 'page', 'help', 'nuneaton', 'gary',
                'company', 'duties','references', 'pas', 'jobsekers',
                'areas', 'relate', 'key', 'request', 'resume', 'competencies', 'ambitious', 'email',
                'driving', 'yes', 'restaurant', 'vyse', 'sekers', 'suitable', 'expertise', 'level',
                'qualifications', 'team', 'understand', 'pose', 'god', 'experience', 'maner',
                'knowledge', 'license', 'particular', 'highly', 'create', 'confident', 'nationality',
                'write', 'responsible', 'profesional', 'having', 'peg', 'staf', 'wel', 'excite', 
                'project', 'providing','project', 'record', 'environment',
                'train', 'company', 'performance',  'plan', 'implement', 'include', 'staf', 
                'team',  'new', 'resource', 'profesional', 'conduct', 'reduce', 'procedures',
                'relations', 'training', 'executive', 'personel', 'xae', 'senior', 'job', 'result', 
                'candidates', 'isues','department', 'division', 'improve','change', 'identify', 
                'leadership', 'lead', 'provide', 'time', 'work', 'level', 'save',
                'project', 'work','plant','team', 'specifications', 'plan', 'environmental','experience',
                'company', 'knowledge', 'include', 'india','phase', 'client',
                'complete', 'preparation', 'society','wind','scope', 'sub', 'completion', 
                'excelent', 'forte', 'god', 'skils', 'poses', 'key', 'air', 'structure', 'prepare',
                'farm','multitasking', 'agriculture', 'qualification', 'til', 'know', 'horticulture',
                'fluent', 'knowledge', 'profesional', 'template', 'florida', 'latin',
                'date', 'proficient', 'presentations', 'pest', 'gadgets', 'significant',
                'report', 'degre', 'achievement', 'player', 'resume', 'english', 'languages',
                'include', 'behavior', 'interests', 'skiled','various', 'cary', 'adept', 'bachelor', 'com',
                'crop', 'wild', 'extensive', 'stret', 'gmail', 'amigo',
                'nursery', 'supervisor', 'update', 'experience', 'miami', 
                'exceptionaly', 'wel', 'joye', 'diploma', 'harvest', 'won', 
                'mower', 'objective', 'sonata', 'carer', 'thompson', 'study', 'gren', 
                'display', 'understand','new', 'skils', 'qualifications', 'strong', 'open',
                'teler', 'cal', 'procedures', 'ability', 'work', 'knowledge',
                'check', 'phone', 'experience', 'asistant', 'excelent', 'client', 
                'god', 'georgia', 'representative', 'telers', 'daily', 'various', 
                'objective', 'asisted', 'sumary', 'profesional', 'provide', 'request', 'jersey', 
                'parkland', 'personalize', 'cals', 'policies', 'reference', 'cityland',
                'efective', 'position', 'handle', 'center', 'provided', 'carer',
                'handled', 'seking', 'fod', 'com', 'acount', 'email',
                'high', 'orient', 'interpersonal', 'properties', 'order', 'furnish', 'train', 'handling',
                'adept','york', 'present', 'nevada' ,'experience', 'campaign', 'frelance', 'online', 'work',
                'clients', 'xad','include', 'gogle','new', 'mobile', 'create',
                'facebok', 'galery', 'layout', 'university','mac', 'model', 'concept', 'layouts',
                'concepts', 'present', 'monash', 'team', 'york', 'lead', 'created', 
                'california', 'corporate', 'twiter', 'company', 'usability',
                'ideas', 'sample', 'client', 'skils', 'direction','care', 
                'asociation', 'center', 'comunity', 'xad', 'practice', 'present', 
                'university', 'member', 'department', 'terminology', 'eagan', 'staf', 'schol'
                'american', 'provide', 'education', 'public', 'colege', 'new'
                'carolina', 'york', 'conference', 'board', 'group', 
                'state', 'ability', 'experience', 'knowledge', 'kovner', 'perform', 'cordinator', 
                'profesional', 'condition', 'train', 'family', 'home', 'biling', 'registered', 
                'record', 'general', 'seidman', 'work','anual', 'ray']

In [73]:
df_after_3 = remove_domain_stop_words(stop_words = stop_words_2, df_edit = df_after_2)

In [74]:
tfidf3 = TfidfVectorizer(stop_words='english')
doc_words3 = tfidf3.fit_transform(df_after_3.TEXT)
tfidf_df3 = pd.DataFrame(doc_words3.toarray(),columns=tfidf3.get_feature_names())
tfidf_df3

C:\Users\Hamna\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aa,aaa,aaaa,aaaaa,aaaaaa,aaaaaaaa,aaaaaaaaa,aaaaaaaaaa,aaaaaaaaaaa,aaaaaaaaaaaa,...,zwart,zwischen,zwitserloot,zxcxbcrich,zxcxbcrichnand,zxcxbcrichncharles,zxcxbcrichnngrants,zxcxbcrichnpxeexxd,zz,zzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
topic_nums = list(np.arange(10, 20, 1))
for num in topic_nums:
    nmf = NMF(num)
    nmf.fit_transform(doc_words3)
    display_topics(nmf, tfidf3.get_feature_names_out(), 100)
    print("------------------------------------------")

C:\Users\Hamna\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
nxefxxb, customers, generating, analyzing, excellent, tasks, negotiations, included, responsibilitiesnxefxxbnnmajor, promotions, good, price, products, latinnother, englishnxefxxb, nlanguages, comnprofessional, nprofessional, negotiating, nsignificant, updates, analytical, stores, giving, fortenxefxxbnnpossess, neducational, consumer, cannxefxxb, bsales, exceptionally, technological, knownnxefxxbnnfluent, qualificationnxefxxbnnbachelors, namigo, skillsnxefxxb, performing, nxxxxxgmail, till, including, interestsnxefxxbnncarry, datensales, repnkey, thompsonn, qualificationsnxefxxb, nsales, strategy, nprice, town, needs, streetnmiami, process, nhome, fln, developing, seminars, bids, promoting, nresponsible, representativenkey, nnxcxefxxb, cell, trip, nxxxgmail, equipment, presentnxefxxb, creating, programs, strategies, achievementnwon, nxxxxgmail, agreements, experiencejob, analystn, detailing, achievementnnxefxxbnnwon, templatennjoye, streetnnewland, qualificationnnxefxxbnnbach

C:\Users\Hamna\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
nxefxxb, customers, generating, analyzing, excellent, tasks, negotiations, responsibilitiesnxefxxbnnmajor, included, promotions, good, price, products, latinnother, englishnxefxxb, nlanguages, comnprofessional, nprofessional, negotiating, nsignificant, updates, analytical, stores, giving, fortenxefxxbnnpossess, neducational, consumer, cannxefxxb, bsales, exceptionally, knownnxefxxbnnfluent, technological, qualificationnxefxxbnnbachelors, namigo, skillsnxefxxb, performing, nxxxxxgmail, till, including, interestsnxefxxbnncarry, datensales, repnkey, thompsonn, qualificationsnxefxxb, nsales, nprice, strategy, town, needs, streetnmiami, nhome, process, fln, developing, promoting, representativenkey, nnxcxefxxb, nresponsible, trip, cell, bids, presentnxefxxb, nxxxgmail, seminars, programs, creating, achievementnwon, agreements, nxxxxgmail, strategies, analystn, achievementnnxefxxbnnwon, templatennjoye, streetnnewland, qualificationnnxefxxbnnbachelors, equipment, sampling, detailing

C:\Users\Hamna\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


...


Topic  0
nxefxxb, generating, analyzing, customers, excellent, tasks, negotiations, responsibilitiesnxefxxbnnmajor, included, promotions, price, latinnother, good, englishnxefxxb, nlanguages, comnprofessional, nprofessional, products, negotiating, nsignificant, updates, stores, analytical, fortenxefxxbnnpossess, giving, neducational, cannxefxxb, knownnxefxxbnnfluent, qualificationnxefxxbnnbachelors, technological, exceptionally, bsales, consumer, namigo, skillsnxefxxb, nxxxxxgmail, performing, including, interestsnxefxxbnncarry, till, repnkey, datensales, thompsonn, qualificationsnxefxxb, nprice, streetnmiami, nsales, strategy, fln, process, town, nhome, needs, developing, nnxcxefxxb, nresponsible, representativenkey, presentnxefxxb, promoting, bids, trip, nxxxgmail, cell, seminars, creating, achievementnwon, analystn, nxxxxgmail, agreements, streetnnewland, templatennjoye, achievementnnxefxxbnnwon, qualificationnnxefxxbnnbachelors, programs, sampling, strategies, detailing, templaten

C:\Users\Hamna\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
nxefxxb, customers, generating, analyzing, excellent, tasks, negotiations, responsibilitiesnxefxxbnnmajor, included, promotions, price, products, good, latinnother, englishnxefxxb, nlanguages, comnprofessional, nprofessional, negotiating, nsignificant, updates, stores, analytical, fortenxefxxbnnpossess, giving, neducational, consumer, cannxefxxb, knownnxefxxbnnfluent, exceptionally, bsales, qualificationnxefxxbnnbachelors, technological, namigo, skillsnxefxxb, performing, nxxxxxgmail, interestsnxefxxbnncarry, including, datensales, till, repnkey, thompsonn, qualificationsnxefxxb, nsales, strategy, nprice, needs, streetnmiami, town, fln, nhome, process, developing, representativenkey, promoting, nresponsible, nnxcxefxxb, presentnxefxxb, bids, trip, nxxxgmail, cell, achievementnwon, creating, strategies, programs, nxxxxgmail, agreements, analystn, equipment, achievementnnxefxxbnnwon, streetnnewland, templatennjoye, seminars, qualificationnnxefxxbnnbachelors, detailing, sampling

C:\Users\Hamna\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
nxefxxb, generating, analyzing, customers, excellent, tasks, negotiations, responsibilitiesnxefxxbnnmajor, included, promotions, price, latinnother, good, englishnxefxxb, nlanguages, comnprofessional, products, nprofessional, negotiating, nsignificant, updates, stores, analytical, fortenxefxxbnnpossess, giving, neducational, cannxefxxb, knownnxefxxbnnfluent, qualificationnxefxxbnnbachelors, exceptionally, technological, bsales, consumer, namigo, skillsnxefxxb, nxxxxxgmail, performing, including, interestsnxefxxbnncarry, datensales, repnkey, till, thompsonn, qualificationsnxefxxb, strategy, nprice, streetnmiami, nsales, fln, town, needs, nhome, process, developing, nnxcxefxxb, nresponsible, representativenkey, promoting, presentnxefxxb, bids, trip, nxxxgmail, cell, creating, achievementnwon, analystn, nxxxxgmail, agreements, strategies, seminars, templatennjoye, streetnnewland, achievementnnxefxxbnnwon, qualificationnnxefxxbnnbachelors, programs, equipment, sampling, experienc

In [76]:
nmf = NMF(13)
doc_topic_nmf = nmf.fit_transform(doc_words3)
display_topics(nmf, tfidf3.get_feature_names_out(), 100)

C:\Users\Hamna\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
nxefxxb, generating, customers, analyzing, excellent, tasks, negotiations, responsibilitiesnxefxxbnnmajor, included, promotions, price, good, products, latinnother, englishnxefxxb, nlanguages, comnprofessional, nprofessional, negotiating, nsignificant, updates, stores, analytical, fortenxefxxbnnpossess, giving, neducational, consumer, cannxefxxb, bsales, knownnxefxxbnnfluent, exceptionally, technological, qualificationnxefxxbnnbachelors, namigo, skillsnxefxxb, performing, nxxxxxgmail, till, including, interestsnxefxxbnncarry, datensales, repnkey, thompsonn, qualificationsnxefxxb, nsales, nprice, streetnmiami, town, needs, strategy, fln, process, nhome, developing, representativenkey, promoting, nresponsible, nnxcxefxxb, trip, presentnxefxxb, bids, nxxxgmail, cell, seminars, creating, achievementnwon, programs, nxxxxgmail, agreements, strategies, analystn, achievementnnxefxxbnnwon, templatennjoye, streetnnewland, qualificationnnxefxxbnnbachelors, sampling, detailing, experienc

In [86]:
df_after_3.shape

(1257, 2)

# Data Labeling

In [87]:
topics_of_resume = pd.DataFrame(doc_topic_nmf, columns = ['Business_Administration', 'Sales_and_Marketing' , 
                                                          'Accounting' , 'Management_Information_System' , 
                                                          'Risk_Management' , 'Law' ,
                                                          'Electrical_and_Mechanical_Engineering', 
                                                          'Health' , 'Graphics_Design' , 'Finance' ,'Teaching' , 
                                                          'Development','hospitality'])
topics_of_resume['Topic_Name'] = topics_of_resume.idxmax(axis=1)
topics_of_resume

,Business_Administration,Sales_and_Marketing,Accounting,Management_Information_System,Risk_Management,Law,Electrical_and_Mechanical_Engineering,Health,Graphics_Design,Finance,Teaching,Development,hospitality,Topic_Name
0,0.000000,0.008697,0.000000,0.014055,0.006653,0.003310,0.024989,0.003032,0.090258,0.004661,0.004098,0.000000,0.064446,Graphics_Design
1,0.000000,0.005236,0.000000,0.007929,0.002672,0.000000,0.000000,0.000000,0.010216,0.000000,0.002681,0.000000,0.007407,Graphics_Design
2,0.000957,0.002615,0.000641,0.001609,0.003088,0.000000,0.002199,0.000000,0.035679,0.025605,0.000000,0.024663,0.015660,Graphics_Design
3,0.000674,0.018533,0.001030,0.041819,0.002782,0.000000,0.001742,0.000000,0.055123,0.000000,0.014771,0.005212,0.095702,hospitality
4,0.000000,0.000000,0.000000,0.000962,0.000000,0.000000,0.000000,0.000000,0.019586,0.000000,0.000000,0.000000,0.000000,Graphics_Design
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,0.000000,0.000900,0.000000,0.000000,0.026945,0.014721,0.006585,0.000000,0.000000,0.017832,0.005155,0.048813,0.000000,Development
1253,0.001167,0.000000,0.000000,0.000000,0.000000,0.000287,0.000000,0.000000,0.000000,0.012673,0.000000,0.122075,0.000000,Development
1254,0.000786,0.000000,0.000000,0.000000,0.000000,0.001056,0.000000,0.000000,0.000000,0.013257,0.000000,0.170425,0.000000,Development
1255,0.000000,0.011364,0.000000,0.000000,0.036696,0.000000,0.000000,0.001671,0.000000,0.000000,0.000000,0.060086,0.000000,Development


# Store the Labeld Dataset into Pickle

In [88]:
write_pickle('Resumes.pickle', topics_of_resume)